# Exemplo Básico do AutoGen

Neste exemplo de código, irá utilizar o [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework para criar um agente básico.

O objetivo deste exemplo é mostrar-lhe os passos que iremos usar mais tarde nos exemplos de código adicionais ao implementar os diferentes padrões de agentes.


## Importar os Pacotes Necessários de Python


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Criar o Cliente

Neste exemplo, utilizaremos os [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para aceder ao LLM.

O `model` está definido como `gpt-4o-mini`. Experimente alterar o modelo para outro disponível no marketplace do GitHub Models para observar resultados diferentes.

Como um teste rápido, vamos executar um prompt simples - `Qual é a capital de França`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definir o Agente

Agora que configurámos o `client` e confirmámos que está a funcionar, vamos criar um `AssistantAgent`. Cada agente pode ter atribuído:  
**name** - Um nome curto que será útil para referenciá-lo em fluxos com múltiplos agentes.  
**model_client** - O cliente que criaste no passo anterior.  
**tools** - Ferramentas disponíveis que o Agente pode usar para completar uma tarefa.  
**system_message** - O metaprompt que define a tarefa, o comportamento e o tom do LLM.  

Podes alterar a system message para ver como o LLM responde. Vamos abordar as `tools` na Aula #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Executar o Agente

A função abaixo irá executar o agente. Utilizamos o método `on_message` para atualizar o estado do Agente com a nova mensagem.

Neste caso, atualizamos o estado com uma nova mensagem do utilizador, que é `"Planeia-me umas ótimas férias ensolaradas"`.

Pode alterar o conteúdo da mensagem para ver como o LLM responde de forma diferente.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante ter em conta que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes da utilização desta tradução.
